In [1]:
import mbuild as mb
import foyer
import hoomd
from mbuild.formats.hoomd_forcefield import create_hoomd_forcefield

In [5]:
esp_ff = foyer.Forcefield(forcefield_files="/Users/madilyn/Projects/repos/forcefields/CPDT.xml")

# We have to add the underscore to the names manually if we are using foyer XML files without SMARTS definitions
CPDT = mb.load("/Users/madilyn/Projects/repos/forcefields/CPDT_typed.mol2")

for p in CPDT.particles():
    p.name = f"_{p.name}"
    
box = mb.fill_box(compound=CPDT, n_compounds=5, box=[10,10,10])
box_pmd = esp_ff.apply(box)

snapshot, forcefield, refs = create_hoomd_forcefield(box_pmd, auto_scale=True, r_cut=2.5)

/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C0
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C4
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C2
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _C5
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/foyer/forcefield.py:665: UserWarning: Non-atomistic element type detected. Creating custom element for _H7
  warnings.warn(
/Users/madilyn/miniconda3/envs/espaloma/

Processing LJ and QQ
Processing 1-4 interactions, adjusting neighborlist exclusions
Processing harmonic bonds
Processing harmonic angles
Processing periodic torsions


/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_H8 pos=([3.6947 4.5025 5.1779]), 1 bonds, id: 6149202176> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_H7 pos=([3.523  4.3033 5.177 ]), 1 bonds, id: 6149202320> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C0 pos=([6.5805 3.439  2.4046]), 4 bonds, id: 6149204384> does not have a charge.They will not be accounted for during this calculation.
  warn(
/Users/madilyn/miniconda3/envs/espaloma/lib/python3.9/site-packages/mbuild/compound.py:400: UserWarning: Some particle of <_C1 pos=([6.6352 3.44   2.5453]), 3 bonds, id: 6149204528> does not ha

In [6]:
#setting cpu and simulation 
cpu = hoomd.device.CPU()
sim = hoomd.Simulation(device=cpu,seed=0)
sim.create_state_from_snapshot(snapshot)

#setting the integrator
kt = 1.2
free_particle = hoomd.filter.Tags(tags=[0,1])  #letting hoomd know which particles to update the positions of
integrator = hoomd.md.Integrator(dt = 0.0001)
nvt = hoomd.md.methods.NVT(filter=hoomd.filter.All(), kT=1.2, tau=0.5) #what is tau?
integrator.forces = forcefield
integrator.methods.append(nvt)
sim.operations.integrator = integrator
sim.state.thermalize_particle_momenta(filter=free_particle, kT=kt)
# Set up GSD writer
gsd_writer = hoomd.write.GSD(
    trigger=hoomd.trigger.Periodic(int(2e2)),
    filename="traj_CPDT.gsd",  #name the output file
    mode="wb"
)
sim.operations.writers.append(gsd_writer)

In [7]:
sim.run(1e4)

*Warning*: charge.pppm: RMS error of 0.206545 is probably too high! 0.20654 0.206545
